# Delta Lake Lab 
## Unit 4: CRUD Support

This lab is powered by Dataproc Serverless Spark.

In the previous unit, we -
1. Create an unpartitioned delta table
2. Created a partitioned delta table called loan_db.loans_by_state_delta
3. Studied the files created & layout in the datalake
4. Learned how to look at delta table details
5. Looked at history (there was not any)
6. Created a manifest file
7. Reviewed entries in the Hive metastore

In this unit, we will learn how to -
1. Delete a record and study the delta log
2. Insert a record and study the delta log
3. Update a record and study the delta log
4. Upsert and study the delta log

### 1. Imports

In [3]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [4]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

22/10/22 23:34:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [5]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-lab


In [6]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-lab


In [7]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  885979867746


In [8]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"

In [9]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json

gs://dll-data-bucket-885979867746/delta-consumable/_symlink_format_manifest/:
gs://dll-data-bucket-885979867746/delta-consumable/_symlink_format_manifest/
gs://dll-data-bucket-885979867746/delta-consumable/_symlink_format_manifest/manifest


### 4. Delete support

In [10]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show()

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used


+----------+-----+
|addr_state|count|
+----------+-----+
|        IA|    1|
+----------+-----+



In [11]:
spark.sql("DELETE FROM loan_db.loans_by_state_delta WHERE addr_state='IA'")

DataFrame[num_affected_rows: bigint]

In [12]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show()

+----------+-----+
|addr_state|count|
+----------+-----+
+----------+-----+



Lets look at the data lake:

In [13]:
# Note how the deleted created a json in the delta log directory
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json

gs://dll-data-bucket-885979867746/delta-consumable/_symlink_format_manifest/:
gs://dll-data-bucket-885979867746/delta-consumable/_symlink_format_manifest/
gs://dll-data-bucket-885979867746/delta-consumable/_symlink_format_manifest/manifest


Lets look at the delta log:

In [14]:
# This is the original log
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000000.json 

{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"56ed94c0-271d-4a3f-b076-4a0aca50c945","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"addr_state\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"count\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1666481414161}}
{"add":{"path":"part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet","partitionValues":{},"size":978,"modificationTime":1666481421062,"dataChange":true,"stats":"{\"numRecords\":51,\"minValues\":{\"addr_state\":\"AK\",\"count\":1},\"maxValues\":{\"addr_state\":\"WY\",\"count\":1},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"commitInfo":{"timestamp":1666481424546,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"1","numOutp

In [15]:
# Note the delete in this log
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000001.json 

{"remove":{"path":"part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet","deletionTimestamp":1666481703134,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":978}}
{"add":{"path":"part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.parquet","partitionValues":{},"size":973,"modificationTime":1666481702921,"dataChange":true,"stats":"{\"numRecords\":50,\"minValues\":{\"addr_state\":\"AK\",\"count\":1},\"maxValues\":{\"addr_state\":\"WY\",\"count\":1},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"commitInfo":{"timestamp":1666481703188,"operation":"DELETE","operationParameters":{"predicate":"[\"(spark_catalog.loan_db.loans_by_state_delta.addr_state = 'IA')\"]"},"readVersion":0,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numCopiedRows":"50","numAddedChangeFiles":"0","executionTimeMs":"5049","numAddedFiles":"1","rewriteTimeMs":"1392","numDeletedRows":"1","scanTimeMs":"3656"},"engineInfo":"A

### 5. Create (Insert) support

In [16]:
spark.sql("INSERT INTO loan_db.loans_by_state_delta VALUES ('IA',222222)")

DataFrame[]

In [17]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show()

+----------+------+
|addr_state| count|
+----------+------+
|        IA|222222|
+----------+------+



In [18]:
# Note how the insert created a new parquet file and in the delta log, yet another json
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json

gs://dll-data-bucket-885979867746/delta-consumable/_symlink_format_manifest/:
gs://dll-data-bucket-885979867746/delta-consumable/_symlink_format_manifest/
gs://dll-data-bucket-885979867746/delta-co

In [19]:
# Lets check for the insert
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000002.json 

{"add":{"path":"part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet","partitionValues":{},"size":725,"modificationTime":1666481715970,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":222222},\"maxValues\":{\"addr_state\":\"IA\",\"count\":222222},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"commitInfo":{"timestamp":1666481716047,"operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"readVersion":1,"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{"numFiles":"1","numOutputRows":"1","numOutputBytes":"725"},"engineInfo":"Apache-Spark/3.3.0 Delta-Lake/2.1.0","txnId":"4bdefb3c-dcc0-43a7-bbd0-aecfa9e34468"}}


### 6. Update support

Lets update a record & see the changes in the delta log directory

In [20]:
spark.sql("UPDATE loan_db.loans_by_state_delta SET count = 11111 WHERE addr_state='IA'")

DataFrame[num_affected_rows: bigint]

In [21]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show()

+----------+-----+
|addr_state|count|
+----------+-----+
|        IA|11111|
+----------+-----+



In [22]:
# Note how the update created a new parquet file and in the delta log, yet another json
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-33f34593-184c-40b8-adfe-73facf9f043f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000

In [23]:
# Lets check for the update
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000003.json 

{"remove":{"path":"part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet","deletionTimestamp":1666481726390,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":725}}
{"add":{"path":"part-00000-33f34593-184c-40b8-adfe-73facf9f043f-c000.snappy.parquet","partitionValues":{},"size":725,"modificationTime":1666481726323,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":11111},\"maxValues\":{\"addr_state\":\"IA\",\"count\":11111},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"commitInfo":{"timestamp":1666481726391,"operation":"UPDATE","operationParameters":{"predicate":"(addr_state#1431 = IA)"},"readVersion":2,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numCopiedRows":"0","numAddedChangeFiles":"0","executionTimeMs":"1463","scanTimeMs":"808","numAddedFiles":"1","numUpdatedRows":"1","rewriteTimeMs":"654"},"engineInfo":"Apache-Spark/3.3.0 Delta-Lake/2.1.0","txnId

### 7. Upsert support

In [24]:
toBeMergedRows = [('IA', 555), ('CA', 12345), ('IN', 6666)]
toBeMergedColumns = ['addr_state', 'count']
toBeMergedDF = spark.createDataFrame(toBeMergedRows, toBeMergedColumns)
toBeMergedDF.createOrReplaceTempView("to_be_merged_table")
toBeMergedDF.orderBy("addr_state").show(3)

+----------+-----+
|addr_state|count|
+----------+-----+
|        CA|12345|
|        IA|  555|
|        IN| 6666|
+----------+-----+



In [25]:
spark.sql("DELETE FROM loan_db.loans_by_state_delta WHERE addr_state='IA'")

DataFrame[num_affected_rows: bigint]

In [26]:
spark.sql("SELECT addr_state,count FROM loan_db.loans_by_state_delta WHERE addr_state in ('IA','CA','IN') ORDER BY addr_state").show()

+----------+-----+
|addr_state|count|
+----------+-----+
|        CA|    1|
|        IN|    1|
+----------+-----+



In [27]:
mergeSQLStatement = "MERGE INTO loan_db.loans_by_state_delta as d USING to_be_merged_table as m ON (d.addr_state = m.addr_state) WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT * "

print(mergeSQLStatement)


MERGE INTO loan_db.loans_by_state_delta as d USING to_be_merged_table as m ON (d.addr_state = m.addr_state) WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT * 


In [28]:
spark.sql(mergeSQLStatement)

22/10/22 23:35:47 ERROR ContextCleaner: Error cleaning shuffle 13
java.lang.NullPointerException: Cannot invoke "org.apache.spark.ShuffleStatus.invalidateSerializedMapOutputStatusCache()" because "shuffleStatus" is null
	at org.apache.spark.MapOutputTrackerMaster.$anonfun$unregisterShuffle$1(MapOutputTracker.scala:882)
	at org.apache.spark.MapOutputTrackerMaster.$anonfun$unregisterShuffle$1$adapted(MapOutputTracker.scala:881)
	at scala.Option.foreach(Option.scala:437)
	at org.apache.spark.MapOutputTrackerMaster.unregisterShuffle(MapOutputTracker.scala:881)
	at org.apache.spark.ContextCleaner.doCleanupShuffle(ContextCleaner.scala:241)
	at org.apache.spark.ContextCleaner.$anonfun$keepCleaning$3(ContextCleaner.scala:202)
	at org.apache.spark.ContextCleaner.$anonfun$keepCleaning$3$adapted(ContextCleaner.scala:195)
	at scala.Option.foreach(Option.scala:437)
	at org.apache.spark.ContextCleaner.$anonfun$keepCleaning$1(ContextCleaner.scala:195)
	at org.apache.spark.util.Utils$.tryOrStopSparkCo

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [29]:
spark.sql("SELECT addr_state,count FROM loan_db.loans_by_state_delta WHERE addr_state in ('IA','CA','IN') ORDER BY addr_state").show()

+----------+-----+
|addr_state|count|
+----------+-----+
|        CA|12345|
|        IA|  555|
|        IN| 6666|
+----------+-----+



In [30]:
# Note how the update created a new parquet file and in the delta log, yet another json
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-33f34593-184c-40b8-adfe-73facf9f043f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-9e3ca1ea-93a4-455a-9988-5e1250281ac0-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-c2a74fad-c560-4efd-8b2e-5a8a7ddcc62e-c000.snappy.parquet

gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/:
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/
gs://dll-data-bucket-885979867746/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-buc

In [31]:
# Lets check for the upsert
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000004.json 

{"remove":{"path":"part-00000-33f34593-184c-40b8-adfe-73facf9f043f-c000.snappy.parquet","deletionTimestamp":1666481743150,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":725}}
{"add":{"path":"part-00000-9e3ca1ea-93a4-455a-9988-5e1250281ac0-c000.snappy.parquet","partitionValues":{},"size":397,"modificationTime":1666481743030,"dataChange":true,"stats":"{\"numRecords\":0,\"minValues\":{},\"maxValues\":{},\"nullCount\":{}}"}}
{"commitInfo":{"timestamp":1666481743150,"operation":"DELETE","operationParameters":{"predicate":"[\"(spark_catalog.loan_db.loans_by_state_delta.addr_state = 'IA')\"]"},"readVersion":3,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numCopiedRows":"0","numAddedChangeFiles":"0","executionTimeMs":"1556","numAddedFiles":"1","rewriteTimeMs":"863","numDeletedRows":"1","scanTimeMs":"693"},"engineInfo":"Apache-Spark/3.3.0 Delta-Lake/2.1.0","txnId":"e58e914e-ccbc-4a38-8499-28caff226c39"}}


### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK